In [82]:
import fastbook
fastbook.setup_book()
from fastai.vision.all import *

In [83]:
# Test to view the behavior of view with -1
p = torch.randn(3, 4)
v = p.view(-1, 12)
p, v

(tensor([[-0.9268, -1.7841,  0.5707,  0.6357],
         [ 0.4419, -0.5767, -0.8411, -0.6639],
         [-1.0307,  1.3681,  1.1727, -0.0177]]),
 tensor([[-0.9268, -1.7841,  0.5707,  0.6357,  0.4419, -0.5767, -0.8411, -0.6639, -1.0307,  1.3681,  1.1727, -0.0177]]))

In [84]:
Path = untar_data(URLs.MNIST_SAMPLE)

In [85]:
threes = (Path/'train'/'3')
sevens = (Path/'train'/'7')

In [86]:
stacked_threes = torch.stack([tensor(Image.open(o)) for o in threes.ls()]).float()
stacked_sevens = torch.stack([tensor(Image.open(o)) for o in sevens.ls()]).float()

In [87]:
stacked_threes.shape, stacked_sevens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

In [88]:
threes_and_sevens = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
threes_and_sevens.shape
labels_threes_and_sevens = tensor([1]*len(stacked_threes) + [0]*len(stacked_sevens)).unsqueeze(1)
labels_threes_and_sevens.shape, threes_and_sevens.shape


(torch.Size([12396, 1]), torch.Size([12396, 784]))

In [89]:
# create a DataLoader and a dataset, dataloader further batches the data for each run
dset = list(zip(threes_and_sevens, labels_threes_and_sevens))
dl = DataLoader(dset, batch_size=64, shuffle=True)

In [90]:
# Initialize weights and biases
def init_weights(x, b):
    w = torch.randn(x).requires_grad_()
    bias = torch.randn(b).requires_grad_()
    return (w,bias)
weights, bias = init_weights(28*28, 1)
weights.shape, bias.shape, bias


(torch.Size([784]), torch.Size([1]), tensor([-0.0439], requires_grad=True))

In [91]:
# define the model
def model(x, w, bias):
    x.shape, w.shape, bias.shape
    return x@w + bias

In [102]:
def loss(preds, labels):
    preds = preds.sigmoid()
    return torch.where(labels == 1, 1-preds, preds).mean()

In [100]:
def train_epoch(dl, w, b):
    for x, y in dl:
        preds = model(x, w, b)
        print(preds.shape, y.shape)
        l = loss(preds, y)
        l.backward()
        with torch.no_grad():
            w -= w.grad * 0.01
            b -= b.grad * 0.01
            w.grad.zero_()
            b.grad.zero_()
        return preds,l

In [125]:
preds,lo = train_epoch(dl, weights, bias)

torch.Size([64]) torch.Size([64, 1])


In [126]:
dl, len(dl.dataset), preds, lo

(<fastai.data.load.DataLoader at 0x234740cce90>,
 12396,
 tensor([ 5337.1240,  3769.4934,  1252.2560,    16.8350, -1498.8574, -1158.2384,  2346.7676,  -483.0139,  2418.7122,   882.8147,  6336.2349,  3323.6724,  1973.5610,  6215.3906,   192.1894,  1920.0161,
          1099.3776,  3283.0073, -2167.6548,  2999.4233,  3791.1953,  -437.2311,  1540.9291, -3464.7803, -1313.1859, -2253.2002, -1253.3960,  3755.3569, -1911.2413,   459.3077,  3793.5293,  1265.2866,
          4401.0176,  3227.8584,  4201.7812,  3844.9172, -1460.2880,   206.8522,   861.3663,  2691.6382,  3167.5559,  1744.1893,  1959.3430,  1493.3798,  3306.4700,  1041.3054,  3790.6538,  3778.6792,
          4552.8979,  3248.5388,  1380.2775,  6189.5552,  3949.2300,   849.2740,  3567.3506,  4588.3584,  3618.4272,  1344.4948,  5532.4980,  3192.4297,  1091.0234, -1519.4922,  2045.4062,  5185.5176],
        grad_fn=<AddBackward0>),
 tensor(0.4609, grad_fn=<MeanBackward0>))